#### Basic navigation

Features:
 - Formatted text
 - Formatted tables including cell bg colour and horizontal merging
 - Targetted image retrieval

In [1]:
import docx_converter as dc
from docx import Document
from bs4 import BeautifulSoup
import os
import pypandoc
import os
import shutil
import json
import re
from collections import Counter

In [58]:
import importlib

dc = importlib.reload(dc)

#### Testing

In [70]:
import importlib

dc = importlib.reload(dc)

docx_path = "data/grovia_Carbon-LITE_Template.docx"
os.path.splitext(os.path.basename(docx_path))[0]
compatible_docx_path = f"app/{os.path.splitext(os.path.basename(docx_path))[0]}.docx"
output_path = f"app/{os.path.splitext(os.path.basename(docx_path))[0]}"
lua_script = "scripts/pandoc/docx_cleanup.lua"

os.makedirs(output_path, exist_ok=True)
allowed_alt_texts = ["timeline"]

os.makedirs(f"{output_path}/{dc.FOLDERS['media']}", exist_ok=True)
os.makedirs(f"{output_path}/{dc.FOLDERS['data']}", exist_ok=True)
os.makedirs(f"{output_path}/{dc.FOLDERS['content']}", exist_ok=True)

dc.check_compatibility(docx_path, compatible_docx_path)

## copy index.html to output_path
shutil.copyfile("scripts/index.html", f"{output_path}/index.html")
if not os.path.exists(f"{output_path}/js"):
    shutil.copytree("scripts/js", f"{output_path}/js")
if not os.path.exists(f"{output_path}/css"):
    shutil.copytree("scripts/css", f"{output_path}/css")


## Styles

## Extract text and table styles
DEFAULT_STYLES = dc.extract_styles(compatible_docx_path)
## save to json
with open(f"{output_path}/{dc.FOLDERS['data']}/styles.json", 'w') as f:
    json.dump(DEFAULT_STYLES, f, indent=2)

## Extract table data and formating that differs from the default styles
tables = dc.extract_table_format(compatible_docx_path, DEFAULT_STYLES)
tables
# ## save to json


Compatible document saved as: app/grovia_Carbon-LITE_Template.docx
para._element is NOT None
para._element is NOT None
para._element is NOT None
para._element is NOT None
para._element is NOT None
para._element is NOT None
Found a drawing element
['keep-icon-psychiatry']
para._element is NOT None
para._element is NOT None
para._element is NOT None
para._element is NOT None


{'table_0': {'headers': [{'text': 'Project Name:',
    'textAlign': 'right',
    'verticalAlign': 'middle'},
   {'text': 'Armidale', 'textAlign': 'left', 'verticalAlign': 'middle'}],
  'rows': [[{'text': 'Location:',
     'bold': True,
     'textAlign': 'right',
     'verticalAlign': 'middle'},
    {'text': 'Armidale, NSW', 'textAlign': 'left', 'verticalAlign': 'middle'}],
   [{'text': 'Project Type:',
     'bold': True,
     'textAlign': 'right',
     'verticalAlign': 'middle',
     'iconHtml': "<span class='material-symbols-outlined'>psychiatry</span>",
     'iconPosition': 'start'},
    {'text': 'Environmental Planting',
     'textAlign': 'left',
     'verticalAlign': 'middle'}],
   [{'text': 'Project Area:',
     'bold': True,
     'textAlign': 'right',
     'verticalAlign': 'middle'},
    {'text': '1804.0 ha', 'textAlign': 'left', 'verticalAlign': 'middle'}],
   [{'text': 'Planting Area:',
     'bold': True,
     'textAlign': 'right',
     'verticalAlign': 'middle'},
    {'text': 

In [ ]:
docx_path = "data/grovia_Carbon-LITE_Template.docx"
os.path.splitext(os.path.basename(docx_path))[0]
compatible_docx_path = f"app/{os.path.splitext(os.path.basename(docx_path))[0]}.docx"
output_path = f"app/{os.path.splitext(os.path.basename(docx_path))[0]}"
lua_script = "scripts/pandoc/docx_cleanup.lua"

os.makedirs(output_path, exist_ok=True)
allowed_alt_texts = ["timeline"]

os.makedirs(f"{output_path}/{dc.FOLDERS['media']}", exist_ok=True)
os.makedirs(f"{output_path}/{dc.FOLDERS['data']}", exist_ok=True)
os.makedirs(f"{output_path}/{dc.FOLDERS['content']}", exist_ok=True)

dc.check_compatibility(docx_path, compatible_docx_path)

## copy index.html to output_path
shutil.copyfile("scripts/index.html", f"{output_path}/index.html")
if not os.path.exists(f"{output_path}/js"):
    shutil.copytree("scripts/js", f"{output_path}/js")
if not os.path.exists(f"{output_path}/css"):
    shutil.copytree("scripts/css", f"{output_path}/css")


## Styles

## Extract text and table styles
DEFAULT_STYLES = dc.extract_styles(compatible_docx_path)
## save to json
with open(f"{output_path}/{dc.FOLDERS['data']}/styles.json", 'w') as f:
    json.dump(DEFAULT_STYLES, f, indent=2)

## Extract table data and formating that differs from the default styles
tables = dc.extract_table_format(compatible_docx_path, DEFAULT_STYLES)
## save to json
with open(f'{output_path}/{dc.FOLDERS['data']}/tables.json', 'w') as f:
    json.dump(tables, f, indent=2)

## End Styles ##
## Images ##

alt_text_map = dc.extract_docx_media(compatible_docx_path, output_path, dc.FOLDERS['media'], allowed_alt_texts)
print("\n📝 Alt Text to Image Mapping:", alt_text_map)
## convert to a list of image integers
# keep_images = [int(name.replace('image', '').replace('.png', '')) for name in alt_text_map.keys()]
keep_images = [value.replace("assets", "media") for _, value in alt_text_map.items()]
print(f"keep_images: {keep_images}")
images_dict = {image: {'path_doc': path.replace('assets/', './media/'), 'path': path, 'alt_text': ''} for image, path in alt_text_map.items()}
print("images_dict: ", images_dict)


## Generate HTML from the docx file
initial_html = dc.convert_docx_to_html(compatible_docx_path, lua_script, keep_images)
print("HTML with unwanted images removed has been generated.")

initial_html_clean = dc.remove_empty_paragraphs(initial_html)

missing_figures = dc.check_for_missing_figures(compatible_docx_path, initial_html_clean)
if missing_figures:
    for missing_figure in missing_figures:
        print(f"⚠️ WARNING: Missing figure: {missing_figure}")

# Identify figure captions and their corresponding images
doc_img_src = [images_dict[img]['path_doc'] for img in images_dict]
figure_captions = dc.get_figure_captions(initial_html_clean, doc_img_src)

# Remove empty <figure> tags
# number of <figure> tags before cleaning
num_figures_before = initial_html_clean.count('<figure>')
# Remove empty <figure> tags
html_captions_removed = dc.remove_empty_figures(initial_html_clean)
# number of <figure> tags after cleaning
num_figures_after = html_captions_removed.count('<figure>')
print(f"Number of <figure> tags before cleaning: {num_figures_before}")
print(f"Number of <figure> tags after cleaning: {num_figures_after}")

image_mapping = dc.find_image_alt_text(html_captions_removed)
# remove keep from alt text
for key, value in image_mapping.items():
    image_mapping[key] = value.replace("keep-", "")

icons_src, charts_src, images_src = dc.identify_image_type(image_mapping)

## Replace icons with placeholders
html_icons = dc.replace_icons_with_placholders(html_captions_removed, icons_src)

## Replace charts with placeholders
print(charts_src)
html_charts = dc.replace_charts_with_placeholders(html_icons, charts_src, figure_captions)

# Replace images with placeholders
html_images = dc.replace_images_with_placeholders(html_charts, images_src, figure_captions)

## Save to json
with open(f'{output_path}/{dc.FOLDERS['data']}/media.json', 'w') as f:
    json.dump(alt_text_map, f, indent=2)

print(f"alt_text_map: {alt_text_map}")

## End Images ##
## Tables ##


# Regular expression to match tables
table_pattern = re.compile(r'<table.*?</table>', re.DOTALL)

# Reset the counter to zero before replacing tables
counter = [0]
html_tables_id = table_pattern.sub(lambda match: dc.table_replacer(match, counter), html_images)

## End Tables ##
## Navigation ##

# Generate navigation data
nav_data = dc.generate_navigation_data(html_tables_id)

# Save navigation data to JSON file
with open(f"{output_path}/{dc.FOLDERS['data']}/navigation.json", "w", encoding="utf-8") as f:
    json.dump(nav_data, f, indent=4)

# Embed navigation data placeholder as the first element in the HTML
if '<div data-navigation></div>' not in html_tables_id:
    content_html = '<div data-navigation></div>\n' + html_tables_id

# Embed sub navigation data placeholder in the HTML. Place at the line above the first h2 tag of each section
content_html = dc.insert_sub_navigation(content_html, nav_data)

## End Navigation ##

# Save the modified content.html
with open(f"{output_path}/{dc.FOLDERS['content']}/content.html", "w", encoding="utf-8") as f:
    f.write(content_html)

print("Navigation JSON file and content placeholder updated successfully!")
print(f"Conversion complete! HTML file saved as {output_path}.")


Compatible document saved as: app/grovia_Carbon-LITE_Template.docx
📂 Creating media folder: app/grovia_Carbon-LITE_Template\assets
✅ Moving image: image2.png ➝ app/grovia_Carbon-LITE_Template\assets\image2.png
✅ Moving image: image11.png ➝ app/grovia_Carbon-LITE_Template\assets\image11.png
✅ Moving image: image12.png ➝ app/grovia_Carbon-LITE_Template\assets\image12.png
✅ Moving image: image13.png ➝ app/grovia_Carbon-LITE_Template\assets\image13.png
✅ Moving image: image15.png ➝ app/grovia_Carbon-LITE_Template\assets\image15.png
✅ Moving image: image22.png ➝ app/grovia_Carbon-LITE_Template\assets\image22.png

📝 Alt Text to Image Mapping: {'image2': 'assets/image2.png', 'image11': 'assets/image11.png', 'image12': 'assets/image12.png', 'image13': 'assets/image13.png', 'image15': 'assets/image15.png', 'image22': 'assets/image22.png'}
keep_images: ['media/image2.png', 'media/image11.png', 'media/image12.png', 'media/image13.png', 'media/image15.png', 'media/image22.png']
images_dict:  {'ima

In [5]:
initial_html_clean

'<h1 id="overview">Overview</h1>\r\n<p>The following is a property-specific assessment of select variables\r\nthat are important to consider when assessing the feasibility of\r\ndeveloping an <a\r\nhref="What%20is%20an%20Environmental%20Planting%20(EP)">Environmental\r\nPlanting (EP).</a></p>\r\n<p>You will be able to view fundamental information regarding the\r\npotential opportunity and obstacles to project development.\xa0</p>\r\n<p>With a PRO and PRO<sup>+</sup> subscription you will also have access\r\nto more elaborate information and interpretation of additional factors\r\nthat are required for critical review in the decision-making process of\r\nyour land assessment and project development.</p>\r\n<h2 id="summary-of-assessment">Summary of Assessment</h2>\r\n<table>\r\n<colgroup>\r\n<col style="width: 21%" />\r\n<col style="width: 78%" />\r\n</colgroup>\r\n<thead>\r\n<tr>\r\n<th style="text-align: right;">Project Name:</th>\r\n<th>Armidale</th>\r\n</tr>\r\n</thead>\r\n<tbody>\r\

In [79]:
images_src

{'./media/image22.png': 'timeline'}

In [ ]:
from collections import namedtuple

## figure number
def get_figure_captions(html_content, doc_img_src: list) -> namedtuple:
    soup = BeautifulSoup(html_content, 'html.parser')
    figure_tags = soup.find_all('figure')
    figure_number = 0
    figure_number_new = 0
    Image = namedtuple('Image', ["figure_number", "figure_number_new", "figure_caption", "alt_text"])
    result = {}

    for figure in figure_tags:
        figure_number += 1
        if not figure.find('img') or figure.find('img')['src'] not in doc_img_src:
            continue
        figure_number_new += 1
        figure_caption = figure.find('figcaption').text if figure.find('figcaption') else ""
        alt_text = figure.find('img')['alt'] if figure.find('img')['alt'] else ""
        current_img = os.path.splitext(os.path.basename(figure.find('img')['src']))[0]
        result[current_img] = Image(figure_number, figure_number_new, figure_caption, alt_text)

    return result


In [41]:
doc_img_src = [images_dict[img]['path_doc'] for img in images_dict]

In [42]:
doc_img_src

['./media/image2.png',
 './media/image11.png',
 './media/image12.png',
 './media/image13.png',
 './media/image14.png',
 './media/image15.png',
 './media/image16.png',
 './media/image18.png',
 './media/image22.png']

In [80]:
html_images

'<h1 id="overview">Overview</h1>\n<p>The following is a property-specific assessment of select variables\r\nthat are important to consider when assessing the feasibility of\r\ndeveloping an <a href="What%20is%20an%20Environmental%20Planting%20(EP)">Environmental\r\nPlanting (EP).</a></p>\n<p>You will be able to view fundamental information regarding the\r\npotential opportunity and obstacles to project development.\xa0</p>\n<p>With a PRO and PRO<sup>+</sup> subscription you will also have access\r\nto more elaborate information and interpretation of additional factors\r\nthat are required for critical review in the decision-making process of\r\nyour land assessment and project development.</p>\n<h2 id="summary-of-assessment">Summary of Assessment</h2>\n<table>\n<colgroup>\n<col style="width: 21%"/>\n<col style="width: 78%"/>\n</colgroup>\n<thead>\n<tr>\n<th style="text-align: right;">Project Name:</th>\n<th>Armidale</th>\n</tr>\n</thead>\n<tbody>\n<tr>\n<td style="text-align: right;"><

In [66]:
out

{'image11': Image(figure_number=8, figure_number_new=1, figure_caption='. Precipitation', alt_text='keep-chart-precipitation'),
 'image12': Image(figure_number=9, figure_number_new=2, figure_caption='. Temperature', alt_text='keep-chart-temperature'),
 'image13': Image(figure_number=10, figure_number_new=3, figure_caption='. Frost', alt_text='keep-chart-frost'),
 'image15': Image(figure_number=11, figure_number_new=4, figure_caption='. Standardised Precipitation Evapotranspiration Index\r\n(SPEI-6) over the last 20 years for the project site', alt_text='keep-chart-drought'),
 'image22': Image(figure_number=16, figure_number_new=5, figure_caption='. A timeline outlining the key stages of a carbon\r\nproject, from initial desktop assessment and site visits to CER project\r\nregistration and ongoing monitoring, reporting, and verification\r\npost-planting.', alt_text='keep-timeline')}

In [ ]:
def find_image_alt_text(html_content):
    # Parse HTML
    soup = BeautifulSoup(html_content, 'html.parser')

    # create a named tuple to store the image filename and alt text
    Image = collections.namedtuple('Image', ['path_doc', 'path_new', 'alt_text', 'figure_number_doc', 'figure_number_new', 'caption'])

    # Map image information to a list of named tuples
    image_mapping = {}

    for 

    # Find all image tags in the HTML
    for img in soup.find_all('img'):
        # Check if the image tag has a parent figure tag
        figure = img.find_parent('figure')

        # Get the image filename and alt text
        filename = img['src']
        alt_text = img.get('alt', '')
        caption = figure.find('figcaption').text if figure and figure.find('figcaption') else ''

        # Store the image information in the named tuple
        image_mapping[filename] = Image(filename, alt_text, figure.number if figure else None, caption)

    # Create a mapping of image filename to alt text from the HTML
    html_alt_map = {
        img['src']: img.get('alt', '')
        for img in soup.find_all('img') if 'src' in img.attrs
    }

    return html_alt_map

{'image2': 'assets/image2.png',
 'image11': 'assets/image11.png',
 'image12': 'assets/image12.png',
 'image13': 'assets/image13.png',
 'image14': 'assets/image14.png',
 'image15': 'assets/image15.png',
 'image16': 'assets/image16.png',
 'image18': 'assets/image18.png',
 'image22': 'assets/image22.png'}

In [6]:
image_mapping

{'./media/image2.png': 'icon-project_type',
 './media/image11.png': 'chart-precipitation',
 './media/image12.png': 'chart-temperature',
 './media/image13.png': 'chart-frost',
 './media/image14.png': 'icon-drought',
 './media/image15.png': 'chart-drought',
 './media/image16.png': 'icon-bushfire',
 './media/image18.png': 'icon-flood',
 './media/image22.png': 'timeline'}

In [ ]:
# Store alt text and caption
alt_text_map[img_id]['alt_text'] = alt_text.removeprefix(ALT_TEXT_KEEP_PREFIX) if alt_text else ""
alt_text_map[img_id]['figcaption'] = figcaption_text

{'./media/image2.png': 'Project-Type',
 './media/image11.png': 'chart-precipitation',
 './media/image12.png': 'chart-temperature',
 './media/image13.png': 'chart-frost',
 './media/image14.png': 'icon-drought',
 './media/image15.png': 'chart-drought',
 './media/image16.png': 'icon-bushfire',
 './media/image18.png': 'icon-flood',
 './media/image22.png': 'timeline'}

In [6]:
images_dict

{'image2': {'path': 'assets/image2.png', 'alt_text': ''},
 'image11': {'path': 'assets/image11.png',
  'alt_text': 'chart-precipitation',
  'figcaption': '<p>Figure 1. Precipitation</p>'},
 'image12': {'path': 'assets/image12.png',
  'alt_text': 'chart-temperature',
  'figcaption': '<p>Figure 2. Temperature</p>'},
 'image13': {'path': 'assets/image13.png',
  'alt_text': 'chart-frost',
  'figcaption': '<p>Figure 3. Frost</p>'},
 'image14': {'path': 'assets/image14.png', 'alt_text': ''},
 'image15': {'path': 'assets/image15.png',
  'alt_text': 'chart-drought',
  'figcaption': '<p>Figure 4. Standardised Precipitation Evapotranspiration Index\r\n(SPEI-6) over the last 20 years for the project site</p>'},
 'image16': {'path': 'assets/image16.png', 'alt_text': ''},
 'image18': {'path': 'assets/image18.png', 'alt_text': ''},
 'image22': {'path': 'assets/image22.png',
  'alt_text': 'timeline',
  'figcaption': '<p>Figure 5. A timeline outlining the key stages of a carbon\r\nproject, from initia

In [5]:
html_captions_removed

'<h1 id="overview">Overview</h1>\r\n<p>The following is a property-specific assessment of select variables\r\nthat are important to consider when assessing the feasibility of\r\ndeveloping an <a\r\nhref="What%20is%20an%20Environmental%20Planting%20(EP)">Environmental\r\nPlanting (EP).</a></p>\r\n<p>You will be able to view fundamental information regarding the\r\npotential opportunity and obstacles to project development.\xa0</p>\r\n<p>With a PRO and PRO<sup>+</sup> subscription you will also have access\r\nto more elaborate information and interpretation of additional factors\r\nthat are required for critical review in the decision-making process of\r\nyour land assessment and project development.</p>\r\n<h2 id="summary-of-assessment">Summary of Assessment</h2>\r\n<table>\r\n<colgroup>\r\n<col style="width: 21%" />\r\n<col style="width: 78%" />\r\n</colgroup>\r\n<thead>\r\n<tr>\r\n<th style="text-align: right;">Project Name:</th>\r\n<th>Armidale</th>\r\n</tr>\r\n</thead>\r\n<tbody>\r\

In [4]:
html_images

'<h1 id="overview">Overview</h1>\r\n<p>The following is a property-specific assessment of select variables\r\nthat are important to consider when assessing the feasibility of\r\ndeveloping an <a\r\nhref="What%20is%20an%20Environmental%20Planting%20(EP)">Environmental\r\nPlanting (EP).</a></p>\r\n<p>You will be able to view fundamental information regarding the\r\npotential opportunity and obstacles to project development.\xa0</p>\r\n<p>With a PRO and PRO<sup>+</sup> subscription you will also have access\r\nto more elaborate information and interpretation of additional factors\r\nthat are required for critical review in the decision-making process of\r\nyour land assessment and project development.</p>\r\n<h2 id="summary-of-assessment">Summary of Assessment</h2>\r\n<table>\r\n<colgroup>\r\n<col style="width: 21%" />\r\n<col style="width: 78%" />\r\n</colgroup>\r\n<thead>\r\n<tr>\r\n<th style="text-align: right;">Project Name:</th>\r\n<th>Armidale</th>\r\n</tr>\r\n</thead>\r\n<tbody>\r\

In [ ]:
# Convert DOCX to HTML
html_initial2 = pypandoc.convert_file(
    compatible_docx_path, 
    "html", 
    extra_args=[
        "--quiet",
        "--extract-media=."  # Extract media to the current directory
    ]
)

In [33]:
html_initial2

'<p><img src="./media/image1.png"\r\nstyle="width:8.26217in;height:11.68122in" /></p>\r\n<p>Carbon PRO Report</p>\r\n<p>Environmental Planting</p>\r\n<p>Armidale, NSW</p>\r\n<p>12 March 2025</p>\r\n<h1 id="overview">Overview</h1>\r\n<p>The following is a property-specific assessment of select variables\r\nthat are important to consider when assessing the feasibility of\r\ndeveloping an <a\r\nhref="What%20is%20an%20Environmental%20Planting%20(EP)">Environmental\r\nPlanting (EP).</a></p>\r\n<p>You will be able to view fundamental information regarding the\r\npotential opportunity and obstacles to project development.\xa0</p>\r\n<p>With a PRO and PRO<sup>+</sup> subscription you will also have access\r\nto more elaborate information and interpretation of additional factors\r\nthat are required for critical review in the decision-making process of\r\nyour land assessment and project development.</p>\r\n<h2 id="summary-of-assessment">Summary of Assessment</h2>\r\n<table>\r\n<colgroup>\r\n<c

In [30]:
metadata_json = dc.generate_lua_lookup_table(keep_images)

In [31]:
# Convert DOCX to HTML
html_initial = pypandoc.convert_file(
    compatible_docx_path, 
    "html", 
    extra_args=[
        "--quiet",
        f"--lua-filter={lua_script}",  # Replace with your actual Lua filter file
        "--extract-media=.",  # Extract media to the current directory
        "--metadata", f"keep_images={metadata_json}"  # Pass as JSON
    ]
)

In [32]:
html_initial

'<h1 id="overview">Overview</h1>\r\n<p>The following is a property-specific assessment of select variables\r\nthat are important to consider when assessing the feasibility of\r\ndeveloping an <a\r\nhref="What%20is%20an%20Environmental%20Planting%20(EP)">Environmental\r\nPlanting (EP).</a></p>\r\n<p>You will be able to view fundamental information regarding the\r\npotential opportunity and obstacles to project development.\xa0</p>\r\n<p>With a PRO and PRO<sup>+</sup> subscription you will also have access\r\nto more elaborate information and interpretation of additional factors\r\nthat are required for critical review in the decision-making process of\r\nyour land assessment and project development.</p>\r\n<h2 id="summary-of-assessment">Summary of Assessment</h2>\r\n<table>\r\n<colgroup>\r\n<col style="width: 21%" />\r\n<col style="width: 78%" />\r\n</colgroup>\r\n<thead>\r\n<tr>\r\n<th style="text-align: right;">Project Name:</th>\r\n<th>Armidale</th>\r\n</tr>\r\n</thead>\r\n<tbody>\r\

In [39]:
initial_html

'<h1 id="overview">Overview</h1>\r\n<p>The following is a property-specific assessment of select variables\r\nthat are important to consider when assessing the feasibility of\r\ndeveloping an <a\r\nhref="What%20is%20an%20Environmental%20Planting%20(EP)">Environmental\r\nPlanting (EP).</a></p>\r\n<p>You will be able to view fundamental information regarding the\r\npotential opportunity and obstacles to project development.\xa0</p>\r\n<p>With a PRO and PRO<sup>+</sup> subscription you will also have access\r\nto more elaborate information and interpretation of additional factors\r\nthat are required for critical review in the decision-making process of\r\nyour land assessment and project development.</p>\r\n<h2 id="summary-of-assessment">Summary of Assessment</h2>\r\n<table>\r\n<colgroup>\r\n<col style="width: 21%" />\r\n<col style="width: 78%" />\r\n</colgroup>\r\n<thead>\r\n<tr>\r\n<th style="text-align: right;">Project Name:</th>\r\n<th>Armidale</th>\r\n</tr>\r\n</thead>\r\n<tbody>\r\

In [35]:
initial_html_clean

'<h1 id="overview">Overview</h1>\r\n<p>The following is a property-specific assessment of select variables\r\nthat are important to consider when assessing the feasibility of\r\ndeveloping an <a\r\nhref="What%20is%20an%20Environmental%20Planting%20(EP)">Environmental\r\nPlanting (EP).</a></p>\r\n<p>You will be able to view fundamental information regarding the\r\npotential opportunity and obstacles to project development.\xa0</p>\r\n<p>With a PRO and PRO<sup>+</sup> subscription you will also have access\r\nto more elaborate information and interpretation of additional factors\r\nthat are required for critical review in the decision-making process of\r\nyour land assessment and project development.</p>\r\n<h2 id="summary-of-assessment">Summary of Assessment</h2>\r\n<table>\r\n<colgroup>\r\n<col style="width: 21%" />\r\n<col style="width: 78%" />\r\n</colgroup>\r\n<thead>\r\n<tr>\r\n<th style="text-align: right;">Project Name:</th>\r\n<th>Armidale</th>\r\n</tr>\r\n</thead>\r\n<tbody>\r\

In [52]:
html_captions_removed

'<h1 id="overview">Overview</h1>\r\n<p>This section provides an overview of the property and carbon\r\nassessment.</p>\r\n<h2 id="preview">Preview</h2>\r\n<table>\r\n<caption><p>Table 1. Summary of Assessment</p></caption>\r\n<colgroup>\r\n<col style="width: 50%" />\r\n<col style="width: 50%" />\r\n</colgroup>\r\n<thead>\r\n<tr>\r\n<th>Item assessed</th>\r\n<th>Assessment</th>\r\n</tr>\r\n</thead>\r\n<tbody>\r\n<tr>\r\n<td>Planting Area</td>\r\n<td><strong>Moderate risk.</strong> Lots of risk.</td>\r\n</tr>\r\n<tr>\r\n<td>Carbon returns</td>\r\n<td><strong>High risk.</strong> Only 4.1 ACCUs/ha.</td>\r\n</tr>\r\n<tr>\r\n<td>Topography</td>\r\n<td><strong>Low risk.</strong> Little to no areas with slope greater\r\nthan 15 degrees.</td>\r\n</tr>\r\n</tbody>\r\n</table>\r\n<p>Located in the Dandenong ranges amongst wet sclerophyll forests and\r\ncarbon rich soils. Your dream carbon project awaits…</p>\r\n<figure>\r\n<img src="./media/image2.png" style="width:6.03278in;height:2.85398in"\r\n

In [38]:
keep_images

[2, 14, 16, 18, 22]

In [37]:
alt_text_map

{'image2': {'path': 'assets/image2.png', 'alt_text': ''},
 'image14': {'path': 'assets/image14.png',
  'alt_text': 'chart-frost',
  'figcaption': '<p>Figure 1. Frost</p>'},
 'image16': {'path': 'assets/image16.png',
  'alt_text': 'chart-drought',
  'figcaption': '<p>Figure 2. Standardised Precipitation Evapotranspiration Index\r\n(SPEI-6) over the last 20 years for the project site</p>'},
 'image18': {'path': 'assets/image18.png',
  'alt_text': 'A map of land with yellow and orange dots AI-generated content may be incorrect.',
  'figcaption': '<p>Figure 3. Bushfires in the region</p>'},
 'image22': {'path': 'assets/image22.png',
  'alt_text': 'A map of a land with green squares AI-generated content may be incorrect.',
  'figcaption': '<p>Figure 4. Major Vegetation Groups (MVG)</p>'}}

In [70]:
int(re.sub('image', '', list(alt_text_map.keys())[0])) - 1

1

In [60]:
images_dict

{'image2': {'path': 'assets/image2.png',
  'alt_text': 'location',
  'figcaption': '<p>Figure 1. Location within the region and state. This will\r\nuse a different satellite colour scheme (faded) but for some reason I\r\ncouldn’t get it to work</p>'},
 'image5': {'path': 'assets/image5.png',
  'alt_text': 'accu_breakdown',
  'figcaption': '<p>Figure 5. Accumulation curve of gross ACCUs generated\r\neach year over the 25-year permanence period. The first recommended\r\nreporting period of the permanence period is illustrated by a yellow\r\nstar</p>'},
 'image13': {'path': 'assets/image13.png',
  'alt_text': 'precipitation',
  'figcaption': '<p>Figure 12. Precipitation</p>'},
 'image14': {'path': 'assets/image14.png',
  'alt_text': 'temperature',
  'figcaption': '<p>Figure 13. Temperature</p>'},
 'image15': {'path': 'assets/image15.png',
  'alt_text': 'frost',
  'figcaption': '<p>Figure 14. Frost</p>'},
 'image23': {'path': 'assets/image23.png',
  'alt_text': 'timeline',
  'figcaption': 

In [61]:
html_images

'<h1 id="overview">Overview</h1>\r\n<p>This section provides an overview of the property and carbon\r\nassessment.</p>\r\n<h2 id="preview">Preview</h2>\r\n<table>\r\n<caption><p>Table 1. Summary of Assessment</p></caption>\r\n<colgroup>\r\n<col style="width: 50%" />\r\n<col style="width: 50%" />\r\n</colgroup>\r\n<thead>\r\n<tr>\r\n<th>Item assessed</th>\r\n<th>Assessment</th>\r\n</tr>\r\n</thead>\r\n<tbody>\r\n<tr>\r\n<td>Planting Area</td>\r\n<td><strong>Moderate risk.</strong> Lots of risk.</td>\r\n</tr>\r\n<tr>\r\n<td>Carbon returns</td>\r\n<td><strong>High risk.</strong> Only 4.1 ACCUs/ha.</td>\r\n</tr>\r\n<tr>\r\n<td>Topography</td>\r\n<td><strong>Low risk.</strong> Little to no areas with slope greater\r\nthan 15 degrees.</td>\r\n</tr>\r\n</tbody>\r\n</table>\r\n<p>Located in the Dandenong ranges amongst wet sclerophyll forests and\r\ncarbon rich soils. Your dream carbon project awaits…</p>\r\n<div data-image="assets/image2.png" data-caption="&lt;p&gt;Figure 1. Location within t